# Lab | Hypothesis Testing

**Objective**

Welcome to the Hypothesis Testing Lab, where we embark on an enlightening journey through the realm of statistical decision-making! In this laboratory, we delve into various scenarios, applying the powerful tools of hypothesis testing to scrutinize and interpret data.

From testing the mean of a single sample (One Sample T-Test), to investigating differences between independent groups (Two Sample T-Test), and exploring relationships within dependent samples (Paired Sample T-Test), our exploration knows no bounds. Furthermore, we'll venture into the realm of Analysis of Variance (ANOVA), unraveling the complexities of comparing means across multiple groups.

So, grab your statistical tools, prepare your hypotheses, and let's embark on this fascinating journey of exploration and discovery in the world of hypothesis testing!

**Challenge 1**

In this challenge, we will be working with pokemon data. The data can be found here:

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv

In [1]:
#libraries
import pandas as pd
import scipy.stats as st
import numpy as np



In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv")
df

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...
795,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True
796,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True
797,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True
798,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True


- We posit that Pokemons of type Dragon have, on average, more HP stats than Grass. Choose the propper test and, with 5% significance, comment your findings.

In [3]:
# This is a two-sample mean comparison problem.
# HP is continuous. Groups are independent. Population variances are unknown and sample sizes differ.
# we will use a Welch two-sample t-test (unequal variances)
# our hypothesis say Dragon HP > Grass HP
from scipy.stats import ttest_ind
    
dragon_hp = df.loc[df["Type 1"] == "Dragon", "HP"]
grass_hp  = df.loc[df["Type 1"] == "Grass", "HP"]

# Welch t-test
t_stat, p_two_sided = ttest_ind(dragon_hp, grass_hp, equal_var=False, nan_policy="omit")
# convert to one-sided p-value
p_one_sided = p_two_sided / 2
    
t_stat, p_one_sided

(3.3349632905124063, 0.0007993609745420599)

Our null hypothesis is H₀: μ_dragon ≤ μ_grass
The mean HP of Dragon Pokémon is substantially higher than that of Grass Pokémon.
The t-statistic is positive.
The one-sided p-value is well below 0.05

There is strong statistical evidence to conclude that Dragon-type Pokémon have a higher average HP than Grass-type Pokémon.

- We posit that Legendary Pokemons have different stats (HP, Attack, Defense, Sp.Atk, Sp.Def, Speed) when comparing with Non-Legendary. Choose the propper test and, with 5% significance, comment your findings.


In [4]:
from statsmodels.multivariate.manova import MANOVA

# rename problematic columns for formula usage
df = df.rename(columns={"Sp. Atk": "Sp_Atk", "Sp. Def": "Sp_Def"})

stats_cols = ["HP", "Attack", "Defense", "Sp_Atk", "Sp_Def", "Speed"]

formula = " + ".join(stats_cols) + " ~ Legendary"
maov = MANOVA.from_formula(formula, data=df)

print(maov.mv_test())

                   Multivariate linear model
                                                                
----------------------------------------------------------------
       Intercept         Value  Num DF  Den DF   F Value  Pr > F
----------------------------------------------------------------
          Wilks' lambda  0.0592 6.0000 793.0000 2100.8338 0.0000
         Pillai's trace  0.9408 6.0000 793.0000 2100.8338 0.0000
 Hotelling-Lawley trace 15.8953 6.0000 793.0000 2100.8338 0.0000
    Roy's greatest root 15.8953 6.0000 793.0000 2100.8338 0.0000
----------------------------------------------------------------
                                                                
----------------------------------------------------------------
          Legendary        Value  Num DF  Den DF  F Value Pr > F
----------------------------------------------------------------
             Wilks' lambda 0.7331 6.0000 793.0000 48.1098 0.0000
            Pillai's trace 0.2669 6.0000 793.

Tha means that p≪ 0.05 and we can reject the null hypothesis.
There is statistically significant evidence that Legendary and Non-Legendary Pokémon differ in their joint stat profile (HP, Attack, Defense, Sp.Atk, Sp.Def, Speed).

**Challenge 2**

In this challenge, we will be working with california-housing data. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv")
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


**We posit that houses close to either a school or a hospital are more expensive.**

- School coordinates (-118, 34)
- Hospital coordinates (-122, 37)

We consider a house (neighborhood) to be close to a school or hospital if the distance is lower than 0.50.

Hint:
- Write a function to calculate euclidean distance from each house (neighborhood) to the school and to the hospital.
- Divide your dataset into houses close and far from either a hospital or school.
- Choose the propper test and, with 5% significance, comment your findings.
 

In [6]:
# Defining Euclidean distance
import numpy as np
SCHOOL = (-118, 34)    
HOSPITAL = (-122, 37)

def euclidean_dist(lon, lat, ref_lon, ref_lat):
    return np.sqrt((lon - ref_lon)**2 + (lat - ref_lat)**2)

In [8]:
df["dist_school"] = euclidean_dist(df["longitude"], df["latitude"], *SCHOOL)
df["dist_hospital"] = euclidean_dist(df["longitude"], df["latitude"], *HOSPITAL)
    
threshold = 0.50
df["close"] = (df["dist_school"] < threshold) | (df["dist_hospital"] < threshold)
    
df["close"].value_counts()

close
False    10171
True      6829
Name: count, dtype: int64

In [10]:
# Dividing our dataset into houses close and far from either a hospital or school.
close_prices = df.loc[df["close"], "median_house_value"].dropna()
far_prices   = df.loc[df["close"], "median_house_value"].dropna()
    
close_prices.mean(), far_prices.mean(), close_prices.shape[0], far_prices.shape[0]

(246951.98213501245, 246951.98213501245, 6829, 6829)

In [11]:
from scipy.stats import ttest_ind
t_stat, p_two_sided = ttest_ind(close_prices, far_prices, equal_var=False)
# one-sided p-value for H1: mean(close) > mean(far)
p_one_sided = p_two_sided / 2 if t_stat > 0 else 1 - (p_two_sided / 2)
t_stat, p_one_sided

(0.0, 0.5)

In [18]:
school_coords = (-118, 34)
hospital_coords = (-122, 37)
# Euclidean distance between two points (x1, y1) and (x2, y2)
def euclidean_distance(x1, y1, x2, y2):
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
    
# Calculate distances
df["dist_to_school"] = euclidean_distance(df["longitude"], df["latitude"],school_coords[0], school_coords[1])
df["dist_to_hospital"] = euclidean_distance(df["longitude"], df["latitude"],hospital_coords[0], hospital_coords[1])
    
# Display first rows
print(df[["longitude", "latitude", "dist_to_school", "dist_to_hospital"]].head())

   longitude  latitude  dist_to_school  dist_to_hospital
0    -114.31     34.19        3.694888          8.187319
1    -114.47     34.40        3.552591          7.966235
2    -114.56     33.69        3.453940          8.143077
3    -114.57     33.64        3.448840          8.154416
4    -114.57     33.57        3.456848          8.183508


In [19]:
#Define close vs far (close to either school OR hospital)
df["close_to_facility"] = ((df["dist_to_school"] < 0.50) | (df["dist_to_hospital"] < 0.50))
    
# Split the dataset
houses_close = df[df["close_to_facility"]]
houses_far = df[df["close_to_facility"]]

# Check sizes
print("Houses close to school or hospital:", len(houses_close))
print("Houses far from both:", len(houses_far))

Houses close to school or hospital: 6829
Houses far from both: 6829


In [21]:
from scipy import stats
price_close = df[df["close_to_facility"]]["median_house_value"]
price_far = df[df["close_to_facility"]]["median_house_value"]

# Descriptive statistics
print("Mean price (close):", price_close.mean())
print("Mean price (far):", price_far.mean())
    
# Welch two-sample t-test
t_stat, p_value = stats.ttest_ind(price_close, price_far, equal_var=False)
print("\nWelch t-test results")
print("t-statistic:", t_stat)
print("p-value:", p_value)

Mean price (close): 246951.98213501245
Mean price (far): 246951.98213501245

Welch t-test results
t-statistic: 0.0
p-value: 1.0


In [22]:
alpha = 0.05
if p_value < alpha:
    print("Conclusion: Reject H0 at 5% significance.")
    print("House prices differ significantly between close and far locations.")
else:
    print("Conclusion: Fail to reject H0 at 5% significance.")
    print("No significant difference in house prices between groups.")

Conclusion: Fail to reject H0 at 5% significance.
No significant difference in house prices between groups.
